In [1]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, \
    WebDriverException
import time

In [2]:
# Flowster forum base URL
baseURL = 'https://forum.flowster.app'

driver = webdriver.Chrome(executable_path='C:\chromedriver')

# Open Chrome web client using Selenium and retrieve page source
driver.get(baseURL)
baseHTML = driver.page_source

In [3]:
# Get base HTML text and generate soup object
baseSoup = BeautifulSoup(baseHTML, 'html.parser')

In [4]:
# Find all anchor objects that contain category information
categories = baseSoup.find_all('a', class_='category-title-link')

categoryPageURLs=[]
for category in categories:
    categoryPageURL = baseURL + category['href']
    categoryPageURLs.append(categoryPageURL)
    print(categoryPageURL)
    
    
#print(categoryPageURLs)    
    #driver.get(categoryPageURL)

https://forum.flowster.app/c/human-resources/46
https://forum.flowster.app/c/amazon-specific/17
https://forum.flowster.app/c/flowster/2
https://forum.flowster.app/c/product-sourcing/8
https://forum.flowster.app/c/software-tools/26
https://forum.flowster.app/c/fulfillment/36
https://forum.flowster.app/c/traffic/13
https://forum.flowster.app/c/management/32
https://forum.flowster.app/c/misc-topics/55
https://forum.flowster.app/c/financial-management/21
https://forum.flowster.app/c/sales-channels/7
https://forum.flowster.app/c/store-website-management/40


In [5]:
### GET ALL LINKS FOR TOPICS ###


topicPageURLs=[]

for j in range(0, len(categoryPageURLs)):
    
    categoryPageURL = categoryPageURLs[j]
    # Get category HTML text
    driver.get(categoryPageURL)


    # Load the entire webage by scrolling
    lastHeight = driver.execute_script("return document.body.scrollHeight")
    while (True):
        # Scroll to bottom of page
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait for new page segment to load
        time.sleep(0.5)

        # Calculate new scroll height and compare with last scroll height
        newHeight = driver.execute_script("return document.body.scrollHeight")
        if newHeight == lastHeight:
            break
        lastHeight = newHeight


    # Generate soup object
    categoryHTML = driver.page_source
    categorySoup = BeautifulSoup(categoryHTML, 'html.parser')

    # Find all anchor objects that contain topic information
    topicAnchors = categorySoup.find_all('a', class_='title raw-link raw-topic-link')

    # Get hyperlink reference and append it to the base URL to get the topic page URL
    for topic in topicAnchors:
        topicPageURL = baseURL + topic['href']
        topicPageURLs.append(topicPageURL)

#print(topicPageURLs)
    


In [6]:
len(topicPageURLs)

156

In [7]:
### ALL SCRAPING FUNCTIONS -- RETURN AS DICTIONARIES ###


def category(url):
    category = tag(url)[0]
    return(category)


## Leading comment and author (done!)
def leading_comment(url):
    leading_comment = requests.get(url)
    soup2 = BeautifulSoup(leading_comment.text, 'html.parser')
    comments = soup2.find_all("meta", property='og:description')
    lead='No-Text'
    for comment in comments:
        lead = comment.get('content')
        #print('Leading Commnet: ', lead)

    authors = soup2.find_all('span', class_='creator')
    author1 = authors[0].getText()
    #print('Author1: ', author1.strip())
    
    ## convert to a dictionary:
    d = dict()
    d[lead.strip()] = author1.strip()
    return d
    
    
    
## Other comments (count from 2) and their authors (done!)
def other_comment(url):
    others = requests.get(url)
    soup2 = BeautifulSoup(others.text, 'html.parser')
    comments = soup2.findAll('div', {"class": "post"})
    authors = soup2.find_all('span', class_='creator')
    
    d = dict()
    for i in range(1, len(comments)):
        comment = comments[i].getText()
        author = authors[i].getText()
        #print('Comment ' + str(i+1) + ': ' + comment.strip())
        #print('Author: ' + str(i+1) + ': ' + author.strip())
        #print('---------------------')
        
        d.update( {comment.strip() : author.strip()} )
    return d
    
  

## Tags  (done!)
def tag(url):
    tags = requests.get(url)
    soup2 = BeautifulSoup(tags.text, 'html.parser')
    tags = soup2.find_all('span', class_="category-name")
    i = 1
    tag_list=[]
    for g in tags:
        tag = g.getText()
        #print('Tag: ' + str(i) + ': ' + tag)
        i += 1
        tag_list.append(tag)
        #return(tag)
        
    return tag_list
        

        
## Topic titles
def topic_title(url):
    topics = requests.get(url)
    soup2 = BeautifulSoup(topics.text, 'html.parser')
    topics = soup2.find_all("meta", property="og:title")
    for t in topics:
        topic = t.get('content')
        #print('Topic Title: ', topic)
        return(topic)
    

## Views (doesn't work right)
def views(url):
    views = requests.get(url)
    soup2 = BeautifulSoup(views.text, 'html.parser')
    views = soup2.find_all('span')
    print(views)
    #print('Views: ', view.getText())
        
        
        
def dictionary_combination(url):
    topic_key = topic_title(url)
    category_val = category(url)
    tag_val = tag(url)
    
    leading_dic = leading_comment(url)
    other_dic = other_comment(url)
    
    
    
    d = dict()
    d[topic_key] = [category_val]
    d[topic_key].append(tag_val)
    d[topic_key].append(leading_dic)
    d[topic_key].append(other_dic)
    
    return(d)

In [8]:
### SAVE ALL DICTIONARIES AS A JSON FILE ###


import json

final_dict = {}

for j in range(0, len(topicPageURLs)):
    url = topicPageURLs[j]
    
    single_dict = dictionary_combination(url)
    final_dict.update(single_dict)
    #print(j)
    
#print(final_dict)

with open('Flowster_Dict.json', 'w') as f:
    json.dump(final_dict, f)

In [19]:
### ALL SCRAPING FUNCTIONS -- RETURN VALUES ###


## Category
def category2(url):
    category = tag(url)[0]
    return(category)


## Leading comment
def leading_comment2(url):
    leading_comment = requests.get(url)
    soup2 = BeautifulSoup(leading_comment.text, 'html.parser')
    comments = soup2.find_all("meta", property='og:description')
    for comment in comments:
        lead = comment.get('content')
        #print('Leading Commnet: ', lead)
        return(lead)
    
    
## Other comments (count from 2)
def other_comment2(url):
    others = requests.get(url)
    soup2 = BeautifulSoup(others.text, 'html.parser')
    comments = soup2.findAll('div', {"class": "post"})
    i = 2
    comment_list=[]
    for c in comments:
        comment = c.getText()
        comment_list.append(comment.strip())
        #print('Comment ' + str(i) + ': ' + comment)
        i += 1
    return(comment_list)

    

## Tags
def tag2(url):
    tags = requests.get(url)
    soup2 = BeautifulSoup(tags.text, 'html.parser')
    tags = soup2.find_all('span', class_="category-name")
    i = 1
    tag_list=[]
    for g in tags:
        tag = g.getText()
        tag_list.append(tag.strip())
        #print('Tag: ' + str(i) + ': ' + tag)
        i += 1
    return(tag_list)

        
## Authors
def author2(url):
    authors = requests.get(url)
    soup2 = BeautifulSoup(authors.text, 'html.parser')
    authors = soup2.find_all('span', class_='creator')
    i = 1
    author_list=[]
    for a in authors:
        author = a.getText()
        author_list.append(author.strip())
        #print('Author: ' + str(i) + ': ' + author.strip())
        i += 1
    return(author_list)

        
## Topic titles
def topic_title2(url):
    topics = requests.get(url)
    soup2 = BeautifulSoup(topics.text, 'html.parser')
    topics = soup2.find_all("meta", property="og:title")
    for t in topics:
        topic = t.get('content')
        #print('Topic Title: ', topic)
        return(topic)

In [21]:
### SAVE AS CSV FILE ###

import pandas as pd

result_df = pd.DataFrame(columns=['Topic Title', 'Category', 'Tags', 'Authors', 'Leading Comment', 'Other Comments'])



## get other topic contents and append to dataframe:
for j in range(0, len(topicPageURLs)):
    url = topicPageURLs[j]
    result_df = result_df.append({'Topic Title': topic_title2(url), 
                                  'Category': category2(url),
                                  'Tags': tag2(url), 
                                  'Authors': author2(url), 
                                  'Leading Comment': leading_comment2(url), 
                                  'Other Comments': other_comment2(url)}, ignore_index=True)
    #print(j)
    
#print(result_df.shape)



In [22]:
result_df.to_csv('flowster_data.csv')

In [23]:
data = pd.read_csv("flowster_data.csv", index_col=0) 

# Preview the first 5 lines of the loaded data 
data.head(10)

,Topic Title,Category,Tags,Authors,Leading Comment,Other Comments
0,About the Human Resources category,Human Resources,['Human Resources'],['Kane'],Have questions about Human Resources? This is ...,['Have questions about Human Resources? This i...
1,VA using PC2 & Revseller,Human Resources,['Human Resources'],"['twentyfoursevenagent', 'Trent-Admin', 'edsut...",My VA reported not having the Amazon Browser i...,['My VA reported not having the Amazon Browser...
2,Training a VA to find brand contact information,Human Resources,['Human Resources'],"['Cameron.B', 'Mitch', 'Cameron.B', 'Cameron.B...","Hello everyone, I’m having trouble helping to ...","['Hello everyone, I’m having trouble helping t..."
3,Performance reviews for Virtual Assistants?,Human Resources,"['Human Resources', 'Performance Reviews']","['jims', 'Mitch']","Does anyone do this? I’ve considered it, but w...","['Does anyone do this? I’ve considered it, but..."
4,Best job sites for virtual assistants?,Human Resources,"['Human Resources', 'Recruiting']","['Blue', 'jims']",Where does everyone find their VAs? I’ve used ...,['Where does everyone find their VAs? I’ve use...
5,Domestic vs Filipino VAs,Human Resources,"['Human Resources', 'Outsourcing']","['dast1983', 'LizD', 'EBF', 'dast1983', 'EBF']",Hey everyone - Need your opinion and experien...,['Hey everyone -\nNeed your opinion and experi...
6,Budgeting for virtual assistants,Human Resources,"['Human Resources', 'Recruiting']","['MWC1', 'Mitch', 'MWC1', 'Mitch']",Doing budgeting for the upcoming year looking ...,['Doing budgeting for the upcoming year lookin...
7,What email should be used for VA software use?,Human Resources,"['Human Resources', 'Managing a Virtual Team']","['WillCompere', 'Mitch', 'WillCompere', 'Mitch']",Does it matter what email you have linked to t...,['Does it matter what email you have linked to...
8,VA-How Would I Locate a VA to enter listings i...,Human Resources,"['Human Resources', 'Managing a Virtual Team']","['Bill', 'Laura', 'twentyfoursevenagent', 'PLW...","Good morning to all, I am new to this forum. ...","['Good morning to all,\nI am new to this forum..."
9,When to use VAs vs employees?,Human Resources,['Human Resources'],"['dast1983', 'Mitch', 'dast1983', 'Ben', 'dast...",Hi - So I’m ready to bring on my second regul...,['Hi -\nSo I’m ready to bring on my second reg...
